In [1]:
import nltk, re, csv
import numpy as np
import pandas as pd
import scipy
# load the category specfic sentiment analysis file
df = pd.read_csv('category_new_update.csv', encoding="Latin1")

In [2]:
new_df =df.loc[:,]

In [3]:
# make a new dataframe with exclusing reviews
new_1_df =new_df[['userId', 'Business_Id','Review_Id','food','service','ambience','cost','misc']]

In [4]:
# group the reviews with respect to review id
busi_rev_agg =new_1_df.groupby(['Business_Id','Review_Id'],as_index=False).sum()
busi_rev_agg.head()

,Business_Id,Review_Id,food,service,ambience,cost,misc
0,--9e1ONYQuAa-CB_Rrw7Tw,-79slJ5_z6FVKG86yQAPaA,11,2,1,0,3
1,--9e1ONYQuAa-CB_Rrw7Tw,-QMNsRIEMeT0B1zEoivd3g,6,1,1,4,0
2,--9e1ONYQuAa-CB_Rrw7Tw,-ogKPaiHquD1m6SxdfjRWQ,-1,1,1,1,2
3,--9e1ONYQuAa-CB_Rrw7Tw,01RayFuJwHLqrQ30SrBP-A,5,1,5,3,0
4,--9e1ONYQuAa-CB_Rrw7Tw,0gjGRGlnAC96FO7ja7kk3A,3,1,1,0,1


In [5]:
# if the overall review for a particular sentence is greater than 0 then
# assign postive sentiment with respect to that category

busi_rev_agg.loc[busi_rev_agg['food']>0,'food']= 1
busi_rev_agg.loc[busi_rev_agg['food']<0,'food']= -1

busi_rev_agg.loc[busi_rev_agg['service']>0,'service']= 1
busi_rev_agg.loc[busi_rev_agg['service']<0,'service']= -1

busi_rev_agg.loc[busi_rev_agg['ambience']>0,'ambience']= 1
busi_rev_agg.loc[busi_rev_agg['ambience']<0,'ambience']= -1

busi_rev_agg.loc[busi_rev_agg['cost']>0,'cost']= 1
busi_rev_agg.loc[busi_rev_agg['cost']<0,'cost']= -1

busi_rev_agg.loc[busi_rev_agg['misc']>0,'misc']= 1
busi_rev_agg.loc[busi_rev_agg['misc']<0,'misc']= -1

In [6]:
# make business table by taking mean of all its reviews for different categories
business_table = busi_rev_agg.groupby(['Business_Id'],as_index=False).mean()

In [7]:
# read input data from reviews data
input_data = pd.read_csv('20PlusReviews_Excluding_Rare_Biz_Users.csv', encoding="Latin1")

In [8]:
# extract only business, user and star rating given by user to restaurant
bus_user_rating = input_data[['Business Id','User Id','User_Stars']]

In [9]:
# unique list of users to iterate over
user_names = bus_user_rating['User Id'].unique()
business_names = bus_user_rating['Business Id'].unique()

In [10]:
print len(user_names)
print len(business_names)

3002
4087


In [11]:
# make empyty dataframe for making user table
user_table = pd.DataFrame(columns = ['User_Id','food','service','ambience','cost','misc'])

In [12]:
count = 0

for i in user_names:
    # get all the restaurant names with the particualr user id
    res_names = np.asarray(bus_user_rating[bus_user_rating['User Id'] == i].loc[:,'Business Id'])
    star_rating = np.asarray(bus_user_rating[bus_user_rating['User Id'] == i].loc[:,'User_Stars'])
    
    #print res_names
    #print star_rating
    
    sum = np.zeros(5)
    total_rating = 0
    # iterate over all the restuarants which user i has reviewed
    for j in range(0,len(res_names)):
        # take weighed average of all its business vectors
        vect = np.array(business_table[business_table['Business_Id'] == res_names[j]].loc[:,'food':'misc'])
        if vect.size!=0:
            weighted_vect = (star_rating[j])*vect[0]
            sum = sum + weighted_vect
            total_rating = total_rating + star_rating[j]
    
    # divide by total sum of ratings
    sum = sum/total_rating
    # save the user vector
    user_table.loc[count]=[i,sum[0],sum[1],sum[2],sum[3],sum[4]]
    count = count+1

In [13]:
user_table.head()
print count

3002


In [14]:
# function for rescale cosine similarity values in range of 5 to 1
def rescale(inp_vector):
    newupper = 5
    newlower = 1
    max_vector= max(inp_vector)
    min_vector = min(inp_vector)
    temp= max_vector - min_vector
    
    rescaled_vector = (inp_vector-min_vector) * 4 / temp + newlower
    rescaled_vector=np.around(rescaled_vector, decimals=4)
    return rescaled_vector

In [15]:
# define cosine similarity matrix(rows = user , cols = business)
cosine_sim_user_business = np.zeros(shape=(len(user_names),len(business_names)))
# for single user
cosine_sim_user = np.zeros(shape=(len(business_names)))

In [18]:
from scipy import spatial

for i in range(0,len(user_names)):
    
    if (i%500==0):
        print i
    user_vector = np.asarray(user_table[user_table['User_Id'] == user_names[i]].loc[:,'food':'misc'])

    # find the user similarity with all the restaurant vectors we have in business table
    for j in range(0,len(business_names)):
        
        bus_vector = np.asarray(business_table[business_table['Business_Id'] == business_names[j]].loc[:,'food':'misc'])
        sim = 1 - spatial.distance.cosine(user_vector[0], bus_vector[0])
        # save cosine similarity to rescale it further
        cosine_sim_user[j]= sim
    
    cosine_sim_user_business[i][:] = rescale(cosine_sim_user) 
    #print cosine_sim_user_business[i]


0
500
1000
1500
2000
2500
3000


In [100]:
# save results in csv file
df_rating = pd.DataFrame(cosine_sim_user_business)
df_rating.to_csv("predicted_rating_df.csv")

In [103]:
# making ground truth table with input data
rating_table_ground = input_data[['Business Id','User Id','User_Stars']]
rating_table_ground = rating_table_ground.pivot(index='User Id', columns='Business Id', values='User_Stars')

In [104]:
# converting dataframe to matrix
rating_table_ground_matrix=rating_table_ground.values

In [105]:
# converting all Nan values to zero if there are any
rating_table_ground_matrix=np.nan_to_num(rating_table_ground_matrix)

In [106]:
# function to calculate rmse 
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [110]:
# calculate rmse for ground truth and predicted ratings
print rmse(cosine_sim_user_business, rating_table_ground_matrix)

1.63208910243
